In [2]:

# ===============================================================
# 📥 NE PAS EXECUTER C'EST UN FICHIER QUI A DEJA été EXECUTER
# ===============================================================
import pandas as pd
import numpy as np

# ===============================================================
# 📥 Lecture du fichier source
# ===============================================================
nom_fichier = 'traitements/statistiques_communes_dvf_2020_2024.csv'
stats = pd.read_csv(nom_fichier, sep=';', low_memory=False)

# Nettoyage de base
stats = stats.dropna(subset=['nom_commune', 'annee'])
stats['annee'] = pd.to_numeric(stats['annee'], errors='coerce')
stats = stats.dropna(subset=['annee'])
stats['annee'] = stats['annee'].astype(int)

# ===============================================================
# 🏙️ Détection et fusion des arrondissements (Paris, Lyon, Marseille)
# ===============================================================
def ville_simplifiee(nom):
    nom = str(nom)
    if nom.lower().startswith("paris"):
        return "Paris"
    elif nom.lower().startswith("lyon"):
        return "Lyon"
    elif nom.lower().startswith("marseille"):
        return "Marseille"
    else:
        return nom

stats['ville'] = stats['nom_commune'].apply(ville_simplifiee)

# On détecte les colonnes numériques (pour les agréger)
colonnes_numeriques = stats.select_dtypes(include=[np.number]).columns.tolist()
colonnes_non_num = [c for c in stats.columns if c not in colonnes_numeriques]

# Agrégation par ville et année :
# - Pour les colonnes numériques → somme (ou moyenne pondérée si applicable)
# - Pour les colonnes non numériques → première valeur (logique pour une donnée descriptive)
def aggreg_flexible(df):
    data = {}
    for col in df.columns:
        if col in colonnes_numeriques:
            if col == 'prix_m2_moyen' and 'nb_transactions' in df.columns:
                data[col] = (df['prix_m2_moyen'] * df['nb_transactions']).sum() / df['nb_transactions'].sum()
            else:
                data[col] = df[col].sum()
        elif col == 'nom_commune':
            data[col] = df['ville'].iloc[0]  # remplace par "Paris", "Lyon" ou "Marseille"
        else:
            data[col] = df[col].dropna().iloc[0] if not df[col].dropna().empty else np.nan
    return pd.Series(data)

agg = (
    stats.groupby(['ville', 'annee'], dropna=False)
    .apply(aggreg_flexible)
    .reset_index(drop=True)
)

# ===============================================================
# 🗺️ Sélection des grandes villes françaises
# ===============================================================
villes_selection = [
    'Paris', 'Lyon', 'Marseille',
    'Toulouse', 'Bordeaux', 'Montpellier',
    'Lille', 'Rennes', 'Strasbourg', 'Grenoble',
]

# On garde uniquement les villes voulues (insensible à la casse)
stats_simplifie = agg[agg['ville'].str.lower().isin([v.lower() for v in villes_selection])].copy()

# ===============================================================
# 💾 Enregistrement du CSV simplifié
# ===============================================================
fichier_sortie = 'statistiques_grandes_villes_2020_2024_toutes_colonnes.csv'
stats_simplifie.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ Fichier simplifié enregistré : {fichier_sortie}")
print(f"Nombre de lignes : {len(stats_simplifie)}")
print(stats_simplifie[['ville', 'annee', 'prix_m2_moyen', 'nb_transactions']].head())

KeyboardInterrupt: 

In [3]:
import pandas as pd

df_departement = pd.read_csv("donnees/population_municipale_departement_france.csv", sep=';', encoding='utf-8')

df_departement = df_departement[df_departement['nom_dep'].isin(['Rhône', 'Paris', 'Hérault', 'Bouches-du-Rhône', 'Isère', 'Gironde', 'Nord', 'Ille-et-Vilaine', 'Haute-Garonne'])]

# Sélection des colonnes
colonnes_depart = ['dep', 'nom_dep', 'p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop']
df_filtre = df_departement[colonnes_depart]

df_pop_long = df_filtre.melt(
    id_vars=['dep', 'nom_dep'],
    value_vars=['p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop'],
    var_name='annee_code',
    value_name='population_municipale'
)

# Nettoyer la colonne "annee" → extraire le nombre
df_pop_long['annee'] = df_pop_long['annee_code'].str.extract(r'p(\d+)_pop').astype(int) + 2000  # ⬅️ on corrige l’année
# Exemple : p16_pop → 2016, p20_pop → 2020, etc.

df_pop_long = df_pop_long[['nom_dep', 'annee', 'population_municipale']]

# Sauvegarde
df_pop_long.to_csv("donnees/population_departement_filtree.csv", sep=';', encoding='utf-8', index=False)

In [1]:
import pandas as pd

df_departement = pd.read_csv("donnees/population_municipale_departement_france.csv", sep=';', encoding='utf-8')

df_departement = df_departement[df_departement['nom_dep'].isin(['Rhône', 'Paris', 'Hérault', 'Bouches-du-Rhône', 'Isère', 'Gironde', 'Nord', 'Ille-et-Vilaine', 'Haute-Garonne'])]

# Sélection des colonnes
colonnes_depart = ['dep', 'nom_dep', 'p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop']
df_filtre = df_departement[colonnes_depart]

df_pop_long = df_filtre.melt(
    id_vars=['dep', 'nom_dep'],
    value_vars=['p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop'],
    var_name='annee_code',
    value_name='population_municipale'
)

# Nettoyer la colonne "annee" → extraire le nombre
df_pop_long['annee'] = df_pop_long['annee_code'].str.extract(r'p(\d+)_pop').astype(int) + 2000  # ⬅️ on corrige l’année
# Exemple : p16_pop → 2016, p20_pop → 2020, etc.

df_pop_long = df_pop_long[['nom_dep', 'annee', 'population_municipale']]

# Sauvegarde
df_pop_long.to_csv("donnees/population_departement_filtree.csv", sep=';', encoding='utf-8', index=False)

In [3]:
import pandas as pd

df = pd.read_csv('donnees/statistiques_grandes_villes_2020_2024_toutes_colonnes.csv', sep=';')
df_etu = pd.read_csv("donnees/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits.csv", sep=';', encoding='utf-8')
df_inciv = pd.read_csv("donnees/incivilites_filtrees.csv", sep=';', encoding='utf-8')
df_dept = pd.read_csv("donnees/population_departement_filtree.csv", sep=';', encoding='utf-8')


df.columns = df.columns.str.strip()


def corriger_annee(row):
    annee = row['annee']
    ville = row['ville']
    if pd.isna(annee):
        return None
    annee = int(annee)
    if ville == 'Lyon' and 20200 <= annee <= 20240:
        return 2020 + (annee - 20200) // 10
    if ville == 'Marseille' and 38380 <= annee <= 38460:
        return {38380: 2020, 38399: 2021, 38418: 2022, 38437: 2023, 38456: 2024}.get(annee)
    if ville == 'Paris' and 46460 <= annee <= 46560:
        return {46460: 2020, 46483: 2021, 46506: 2022, 46529: 2023, 46552: 2024}.get(annee)
    if 2020 <= annee <= 2024:
        return annee
    return None


df['annee_corrigee'] = df.apply(corriger_annee, axis=1)
df = df.dropna(subset=['annee_corrigee'])
df['annee'] = df['annee_corrigee'].astype(int)

df['prix_m2_moyen'] = pd.to_numeric(df['prix_m2_moyen'], errors='coerce')
df['nb_transactions'] = pd.to_numeric(df['nb_transactions'], errors='coerce')
df = df.dropna(subset=['prix_m2_moyen', 'ville'])

villes = sorted(df['ville'].unique())
df_transaction_rendement = df.copy()
print(f"✅ Données prêtes ({len(df)} lignes). Villes disponibles : {', '.join(villes)}")

✅ Données prêtes (45 lignes). Villes disponibles : Bordeaux, Grenoble, Lille, Lyon, Marseille, Montpellier, Paris, Rennes, Toulouse


In [4]:
import os
# ============================================================================
# ÉTAPE 1 : CHARGEMENT AUTOMATIQUE DE TOUS LES FICHIERS
# ============================================================================

# Mapping codes département → noms de villes
mapping_codes = {
    'L1300': 'Marseille',
    'L6900': 'Lyon',
    'L3100': 'Toulouse',
    'L3400': 'Montpellier',
    'L3500': 'Rennes',
    'L3800': 'Grenoble',
    'L5900': 'Lille',
    'L3300': 'Bordeaux',
    'L7501': 'Paris',
}

# Trouver tous les fichiers Base_OP_*.csv dans le dossier
fichiers_loyers = [f for f in os.listdir("./villes/") if f.startswith("Base_OP_") and f.endswith(".csv")]
print(f"   Fichiers trouvés: {len(fichiers_loyers)}")

dfs_loyers = []

for fichier in fichiers_loyers:
    try:
        # Chemin complet du fichier
        chemin_fichier = os.path.join("./villes/", fichier)
        # Extraire code département du nom de fichier
        # Ex: "Base_OP_2024_L3100.csv" → "L3100"
        code_dept = fichier.split('_')[-1].replace('.csv', '')
        # Mapper le code à la ville
        if code_dept in mapping_codes:
            ville = mapping_codes[code_dept]
            # Charger le fichier CSV avec encoding latin1 (au lieu de utf-8)
            df = pd.read_csv(chemin_fichier, sep=";", encoding="latin1")
            df['Ville'] = ville
            dfs_loyers.append(df)
        else:
            print(f"Code {code_dept} non reconnu")

    except Exception as e:
        print(f" Erreur {fichier}: {e}")

# Concaténer tous les fichiers
if len(dfs_loyers) > 0:
    df_loyers = pd.concat(dfs_loyers, ignore_index=True)
else:
    print("\nAucun fichier n'a pu être chargé!")

# ============================================================================
# ÉTAPE 2 : PRÉPARATION DES DONNÉES LOYERS
# ============================================================================

print("🔄 Préparation des données loyers...\n")

# Filtrer : garder seulement les lignes avec Type_habitat (pas NaN/vide)
df_loyers = df_loyers[df_loyers['Type_habitat'].notna()].copy()
df_loyers = df_loyers[df_loyers['Type_habitat'].str.strip() != ''].copy()

print(f"   Lignes filtrées: {len(df_loyers)}")

# Utiliser Type_habitat directement (Appartement, Maison, Ensemble, etc)
df_loyers['Type_bien'] = df_loyers['Type_habitat'].str.strip()

# Sélectionner les colonnes pertinentes
loyers_clean = df_loyers[[
    'Ville', 'Type_bien', 'loyer_median',
    'loyer_mensuel_median', 'surface_moyenne', 'nombre_observations'
]].copy()

print(f"   Colonnes: {list(loyers_clean.columns)}")

# Vérifier les données
print(f"\n   Types de bien ({len(loyers_clean['Type_bien'].unique())}):")
for t in sorted(loyers_clean['Type_bien'].unique()):
    count = len(loyers_clean[loyers_clean['Type_bien'] == t])
    print(f"      ✓ {t:<20} ({count} lignes)")

# ============================================================================
# ÉTAPE 3 : PRIX AU M² (MeilleursAgents) & RENDEMENT — APPARTEMENTS UNIQUEMENT
# ============================================================================

import numpy as np
from pathlib import Path

# 1) Restreindre aux APPARTEMENTS (sans normaliser les villes)
loyers_clean["Type_bien"] = loyers_clean["Type_bien"].astype(str).str.strip()
df_app = loyers_clean[loyers_clean["Type_bien"].str.lower() == "appartement"].copy()
print(f"   Appartements conservés : {len(df_app)} lignes")

# 2) Table de référence prix/m² (MeilleursAgents) – appartements
#    (valeurs extraites de ton tableau : Paris, Lille, Montpellier, Lyon, Marseille,
#     Grenoble, Toulouse, Bordeaux, Strasbourg, Rennes)
prix_m2_ma = {
    "Paris": 9645,
    "Lille": 3408,
    "Montpellier": 3500,
    "Lyon": 4764,
    "Marseille": 3539,
    "Grenoble": 2588,
    "Toulouse": 3527,
    "Bordeaux": 4511,
    "Strasbourg": 3778,
    "Rennes": 3752,
}
df_ref = pd.DataFrame({"Ville": list(prix_m2_ma.keys()),
                       "prix_m2_appartement": list(prix_m2_ma.values())})

# 3) Fusion simple sur 'Ville' (AUCUNE normalisation)
df_app = df_app.merge(df_ref, on="Ville", how="left")

# 4) Cast & contrôles de base
num_cols = ["loyer_mensuel_median", "surface_moyenne", "prix_m2_appartement"]
for c in num_cols:
    if c in df_app.columns:
        df_app[c] = pd.to_numeric(df_app[c], errors="coerce")

# 5) Calculs : prix d'achat estimé & rendement réel
#    - prix_achat_calc = prix_m2_appartement × surface_moyenne
#    - loyer_annuel = loyer_mensuel_median × 12
#    - rendement_% = (loyer_annuel / prix_achat_calc) × 100
df_app["surface_moyenne"] = df_app["surface_moyenne"].fillna(50)  # fallback simple
df_app["prix_achat_calc"] = df_app["prix_m2_appartement"] * df_app["surface_moyenne"]
df_app["loyer_annuel"] = df_app["loyer_mensuel_median"] * 12
df_app["rendement_%"] = (df_app["loyer_annuel"] / df_app["prix_achat_calc"]) * 100

# 6) Colonnes finales (on n’utilise plus prix/surface “médians concaténés”)
cols_finales = [
    "Ville", "Type_bien",
    "loyer_median", "loyer_mensuel_median", "loyer_annuel",
    "surface_moyenne", "prix_m2_appartement", "prix_achat_calc", "rendement_%"
]
df_out = df_app[cols_finales].copy()

# 7) Export
Path("traitements").mkdir(parents=True, exist_ok=True)
out_path = "traitements/loyers_rendement_reel.csv"
df_out.to_csv(out_path, sep=";", encoding="utf-8", index=False)

print("Export réalisé :", out_path)
print("   Lignes :", len(df_out))
display(df_out.head(10))

   Fichiers trouvés: 9
🔄 Préparation des données loyers...

   Lignes filtrées: 10993
   Colonnes: ['Ville', 'Type_bien', 'loyer_median', 'loyer_mensuel_median', 'surface_moyenne', 'nombre_observations']

   Types de bien (2):
      ✓ Appartement          (7207 lignes)
      ✓ Maison               (3786 lignes)
   Appartements conservés : 7207 lignes
Export réalisé : traitements/loyers_rendement_reel.csv
   Lignes : 7207


,Ville,Type_bien,loyer_median,loyer_mensuel_median,loyer_annuel,surface_moyenne,prix_m2_appartement,prix_achat_calc,rendement_%
0,Grenoble,Appartement,"11,3",620.0,7440.0,59.0,2588,152692.0,4.872554
1,Grenoble,Appartement,"15,0",420.0,5040.0,29.0,2588,75052.0,6.715344
2,Grenoble,Appartement,"12,4",561.0,6732.0,46.0,2588,119048.0,5.654862
3,Grenoble,Appartement,"10,7",690.0,8280.0,65.0,2588,168220.0,4.922126
4,Grenoble,Appartement,"9,8",800.0,9600.0,89.0,2588,230332.0,4.167897
5,Grenoble,Appartement,"15,0",420.0,5040.0,29.0,2588,75052.0,6.715344
6,Grenoble,Appartement,"12,4",561.0,6732.0,46.0,2588,119048.0,5.654862
7,Grenoble,Appartement,"10,7",690.0,8280.0,65.0,2588,168220.0,4.922126
8,Grenoble,Appartement,"9,8",800.0,9600.0,89.0,2588,230332.0,4.167897
9,Grenoble,Appartement,"11,8",632.0,7584.0,59.0,2588,152692.0,4.966861
